In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn
import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/03/07 09:40:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/03/07 09:40:03 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/04-deployment/batch/mlruns/1', creation_time=1741340404011, experiment_id='1', last_update_time=1741340404011, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
train_df = pd.read_csv("./data/green_tripdata_2024-01.csv")

In [4]:
train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [8]:
train_df.keys()

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [9]:
train_df["duration"] = pd.to_datetime(train_df["lpep_dropoff_datetime"]) - pd.to_datetime(train_df["lpep_pickup_datetime"])
train_df["duration"] = train_df["duration"].dt.total_seconds() / 60

In [10]:
train_df = train_df[train_df["duration"] < 60]

In [11]:
train_df.loc[:, "PULocationID"] = train_df["PULocationID"].astype(str)
train_df.loc[:, "DOLocationID"] = train_df["DOLocationID"].astype(str)

/tmp/ipykernel_7038/2614523797.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['236' '65' '74' ... '72' '41' '33']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[:, "PULocationID"] = train_df["PULocationID"].astype(str)
/tmp/ipykernel_7038/2614523797.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['239' '170' '262' ... '72' '42' '91']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[:, "DOLocationID"] = train_df["DOLocationID"].astype(str)


In [12]:
feature_df = train_df[["PULocationID", "DOLocationID"]].drop_duplicates().reset_index(drop=True)

In [13]:
feature_df

,PULocationID,DOLocationID
0,236,239
1,65,170
2,74,262
3,74,116
4,74,243
...,...,...
5176,41,37
5177,52,227
5178,82,49
5179,159,242


In [14]:
feature_dicts = feature_df.to_dict('records')

In [15]:
feature_dicts

[{'PULocationID': '236', 'DOLocationID': '239'},
 {'PULocationID': '65', 'DOLocationID': '170'},
 {'PULocationID': '74', 'DOLocationID': '262'},
 {'PULocationID': '74', 'DOLocationID': '116'},
 {'PULocationID': '74', 'DOLocationID': '243'},
 {'PULocationID': '33', 'DOLocationID': '209'},
 {'PULocationID': '74', 'DOLocationID': '238'},
 {'PULocationID': '166', 'DOLocationID': '239'},
 {'PULocationID': '226', 'DOLocationID': '226'},
 {'PULocationID': '7', 'DOLocationID': '129'},
 {'PULocationID': '42', 'DOLocationID': '75'},
 {'PULocationID': '41', 'DOLocationID': '141'},
 {'PULocationID': '130', 'DOLocationID': '196'},
 {'PULocationID': '74', 'DOLocationID': '69'},
 {'PULocationID': '41', 'DOLocationID': '74'},
 {'PULocationID': '55', 'DOLocationID': '210'},
 {'PULocationID': '41', 'DOLocationID': '42'},
 {'PULocationID': '42', 'DOLocationID': '151'},
 {'PULocationID': '255', 'DOLocationID': '255'},
 {'PULocationID': '41', 'DOLocationID': '24'},
 {'PULocationID': '255', 'DOLocationID': 

In [16]:
from sklearn.feature_extraction import DictVectorizer

In [17]:
dv = DictVectorizer()

In [18]:
dv.fit_transform(feature_dicts).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(5181, 450))

In [19]:
dv.get_feature_names_out()

array(['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100',
       'DOLocationID=101', 'DOLocationID=102', 'DOLocationID=106',
       'DOLocationID=107', 'DOLocationID=108', 'DOLocationID=11',
       'DOLocationID=112', 'DOLocationID=113', 'DOLocationID=114',
       'DOLocationID=116', 'DOLocationID=117', 'DOLocationID=118',
       'DOLocationID=119', 'DOLocationID=12', 'DOLocationID=120',
       'DOLocationID=121', 'DOLocationID=122', 'DOLocationID=123',
       'DOLocationID=124', 'DOLocationID=125', 'DOLocationID=126',
       'DOLocationID=127', 'DOLocationID=128', 'DOLocationID=129',
       'DOLocationID=13', 'DOLocationID=130', 'DOLocationID=131',
       'DOLocationID=132', 'DOLocationID=133', 'DOLocationID=134',
       'DOLocationID=135', 'DOLocationID=136', 'DOLocationID=137',
       'DOLocationID=138', 'DOLocationID=139', 'DOLocationID=14',
       'DOLocationID=140', 'DOLocationID=141', 'DOLocationID=142',
       'DOLocationID=143', 'DOLocationID=144', 'DOLocationID=145',
  

520

Get feature matrix

In [20]:
feature_dicts = train_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [21]:
feature_matrix_train = dv.transform(feature_dicts)

# Training

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [23]:
duration = train_df["duration"].to_numpy()

In [24]:
lr = LinearRegression().fit(feature_matrix_train, duration)

In [28]:
# save model
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [29]:
duration_pred = lr.predict(feature_matrix_train)
mse = mean_squared_error(duration, duration_pred)
rmse = np.sqrt(mse)
rmse

np.float64(7.281606147116313)

In [30]:
del train_df

import gc
gc.collect()

1588

In [31]:
val_df = pd.read_csv("./data/green_tripdata_2024-02.csv")

In [34]:
val_df["duration"] = pd.to_datetime(val_df["lpep_dropoff_datetime"]) - pd.to_datetime(val_df["lpep_pickup_datetime"])
val_df["duration"] = val_df["duration"].dt.total_seconds() / 60
val_df = val_df[val_df["duration"] < 60]

val_df.loc[:, "PULocationID"] = val_df["PULocationID"].astype(str)
val_df.loc[:, "DOLocationID"] = val_df["DOLocationID"].astype(str)

/tmp/ipykernel_7038/1525572808.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['75' '43' '75' ... '75' '97' '80']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  val_df.loc[:, "PULocationID"] = val_df["PULocationID"].astype(str)
/tmp/ipykernel_7038/1525572808.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '242' '238' ... '33' '33' '33']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  val_df.loc[:, "DOLocationID"] = val_df["DOLocationID"].astype(str)


In [35]:
feature_dicts = val_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
feature_matrix_val = dv.transform(feature_dicts)

val_duration = val_df["duration"].to_numpy()
duration_pred = lr.predict(feature_matrix_val)
mse = mean_squared_error(val_duration, duration_pred)
rmse = np.sqrt(mse)
rmse

np.float64(7.458980178490645)

Train 2nd model

In [36]:
from sklearn.linear_model import Lasso

In [37]:
with mlflow.start_run():
    
    # setting tags for easier filtering
    mlflow.set_tag("developer", "katie")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.csv")
    
    alpha = 0.1
    
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(feature_matrix_train, duration)

    duration_pred = lr.predict(feature_matrix_val)
    mse = mean_squared_error(val_duration, duration_pred)
    rmse = np.sqrt(mse)

    mlflow.log_metric("rmse", rmse)

    # saving model as an artefact
    mlflow.sklearn.log_model(lr, artifact_path="models_mlflow")

2025/03/07 09:46:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Logging hyper-parameters

In [39]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [41]:
train = xgb.DMatrix(feature_matrix_train, label=duration)
valid = xgb.DMatrix(feature_matrix_val, label=val_duration)

In [31]:
# setting objective function
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            # on training data
            dtrain=train,
            num_boost_round=100,
            # use validation set to control the optimisation algorithm
            # minimise error on validation set
            evals=[(valid, "validation")],
            # stop if no improvements for 50 rounds
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        mse = mean_squared_error(val_duration, y_pred)
        mlflow.log_metric("rmse", mse**0.5)

    return {'loss': rmse, 'status': STATUS_OK}

In [32]:
# search space - range for hyperopt to explore the parameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    # log uniform space
    'learning_rate': hp.loguniform('learning_rate', -3, 0),   # between exp(-3) and exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    # this is a regression problem, so use regression as objective function
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                        | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:55:02] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.04911                                                                                         
[1]	validation-rmse:8.38270                                                                                         
[2]	validation-rmse:8.05224                                                                                         
[3]	validation-rmse:7.65542                                                                                         
[4]	validation-rmse:7.54594                                                                                         
[5]	validation-rmse:7.46376                                                                                         
[6]	validation-rmse:7.38224                                                                                         
[7]	validation-rmse:7.22001                                                                                         
[8]	validation-rmse:7.17593                                     

/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:56:29] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.80400                                                                                         
[1]	validation-rmse:6.10983                                                                                         
[2]	validation-rmse:5.74874                                                                                         
[3]	validation-rmse:5.70845                                                                                         
[4]	validation-rmse:5.67872                                                                                         
[5]	validation-rmse:5.64584                                                                                         
[6]	validation-rmse:5.61140                                                                                         
[7]	validation-rmse:5.60920                                                                                         
[8]	validation-rmse:5.60683                                     

...

[0]	validation-rmse:9.78524                                                                                         
[1]	validation-rmse:9.35365                                                                                         
[2]	validation-rmse:9.00400                                                                                         
[3]	validation-rmse:8.73173                                                                                         
[4]	validation-rmse:8.52716                                                                                         
[5]	validation-rmse:8.25440                                                                                         
[6]	validation-rmse:8.12178                                                                                         
[7]	validation-rmse:7.94251                                                                                         
[8]	validation-rmse:7.85005                                     

/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [12:57:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.84179                                                                                         
[1]	validation-rmse:7.71525                                                                                         
[2]	validation-rmse:7.35061                                                                                         
[3]	validation-rmse:7.14088                                                                                         
[4]	validation-rmse:6.69458                                                                                         
[5]	validation-rmse:6.60813                                                                                         
[6]	validation-rmse:6.55449                                                                                         
[7]	validation-rmse:6.48450                                                                                         
[8]	validation-rmse:6.34333                                     

/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [12:59:31] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.55696                                                                                         
[1]	validation-rmse:6.53377                                                                                         
[2]	validation-rmse:6.27117                                                                                         
[3]	validation-rmse:6.12650                                                                                         
[4]	validation-rmse:6.03520                                                                                         
[5]	validation-rmse:5.83268                                                                                         
[6]	validation-rmse:5.80378                                                                                         
[7]	validation-rmse:5.64389                                                                                         
[8]	validation-rmse:5.62675                                     

In [42]:
# trained params
params = {
    'max_depth': 87,
    'learning_rate': 0.29122519299691013, 
    'reg_alpha': 0.00716725908061934,
    'min_child_weight': 1.1251068416171426,
    'objective': 'reg:linear',
    'seed': 42
}

# using autolog
mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2025/03/07 09:52:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '76688969aea845f1816ffb954e2b1398', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:7.97210
[1]	validation-rmse:7.19983


/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:52:24] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[2]	validation-rmse:6.78191
[3]	validation-rmse:6.56674
[4]	validation-rmse:6.30874
[5]	validation-rmse:6.18206
[6]	validation-rmse:6.12972
[7]	validation-rmse:6.08824
[8]	validation-rmse:6.04158
[9]	validation-rmse:6.02443
[10]	validation-rmse:6.01411
[11]	validation-rmse:5.99181
[12]	validation-rmse:5.97848
[13]	validation-rmse:5.96942
[14]	validation-rmse:5.95672
[15]	validation-rmse:5.95058
[16]	validation-rmse:5.93637
[17]	validation-rmse:5.92488
[18]	validation-rmse:5.92182
[19]	validation-rmse:5.91965
[20]	validation-rmse:5.90720
[21]	validation-rmse:5.90627
[22]	validation-rmse:5.90532
[23]	validation-rmse:5.89990
[24]	validation-rmse:5.89798
[25]	validation-rmse:5.89607
[26]	validation-rmse:5.89560
[27]	validation-rmse:5.89539
[28]	validation-rmse:5.89435
[29]	validation-rmse:5.89207
[30]	validation-rmse:5.89316
[31]	validation-rmse:5.89011
[32]	validation-rmse:5.88905
[33]	validation-rmse:5.88641
[34]	validation-rmse:5.88488
[35]	validation-rmse:5.88275
[36]	validation-rmse:5

KeyboardInterrupt: 

## Saving models

In [43]:
with mlflow.start_run():
    best_params = {
    'max_depth': 87,
    'learning_rate': 0.29122519299691013, 
    'reg_alpha': 0.00716725908061934,
    'min_child_weight': 1.1251068416171426,
    'objective': 'reg:linear',
    'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=300,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    mse = mean_squared_error(val_duration, y_pred)
    mlflow.log_metric("rmse", mse**0.5)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    # log preprocessor
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:7.97210
[1]	validation-rmse:7.19983


/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:52:37] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[2]	validation-rmse:6.78191
[3]	validation-rmse:6.56674
[4]	validation-rmse:6.30874
[5]	validation-rmse:6.18206
[6]	validation-rmse:6.12972
[7]	validation-rmse:6.08824
[8]	validation-rmse:6.04158
[9]	validation-rmse:6.02443
[10]	validation-rmse:6.01411
[11]	validation-rmse:5.99181
[12]	validation-rmse:5.97848
[13]	validation-rmse:5.96942
[14]	validation-rmse:5.95672
[15]	validation-rmse:5.95058
[16]	validation-rmse:5.93637
[17]	validation-rmse:5.92488
[18]	validation-rmse:5.92182
[19]	validation-rmse:5.91965
[20]	validation-rmse:5.90720
[21]	validation-rmse:5.90627
[22]	validation-rmse:5.90532
[23]	validation-rmse:5.89990
[24]	validation-rmse:5.89798
[25]	validation-rmse:5.89607
[26]	validation-rmse:5.89560
[27]	validation-rmse:5.89539
[28]	validation-rmse:5.89435
[29]	validation-rmse:5.89207
[30]	validation-rmse:5.89316
[31]	validation-rmse:5.89011
[32]	validation-rmse:5.88905
[33]	validation-rmse:5.88641
[34]	validation-rmse:5.88488
[35]	validation-rmse:5.88275
[36]	validation-rmse:5

2025/03/07 09:52:59 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/03/07 09:52:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:52:59] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/03/07 09:53:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/opt/conda/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [09:53:13] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file 

## Loading model

In [32]:
logged_model = 'runs:/7ced78e294ba48c9904625c91346cb01/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [33]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 7ced78e294ba48c9904625c91346cb01

In [34]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [35]:
xgboost_model